In [1]:
import polars as pl
import os

#### Склеивание тестового датасета и перевод его в parquet

In [11]:
csv_folder_path = "data"
output_parquet_path = "output_dataset.parquet"

In [ ]:
dataframes = [pl.read_csv(os.path.join(csv_folder_path, file)) for file in os.listdir(csv_folder_path) if file.endswith(".csv")]

In [ ]:
combined_df = pl.concat(dataframes)

In [ ]:
combined_df.describe()

In [ ]:
combined_df.write_parquet(output_parquet_path)

#### Склеивание учебного датасета и перевод его в parquet

In [4]:
data = pl.read_csv("data/train_6.csv")
data.write_parquet("data/train_6.parquet")

In [5]:
del data

In [ ]:
for file in os.listdir(csv_folder_path):
    if file.endswith(".csv"):
        data = pl.read_csv(os.path.join(csv_folder_path, file))
        data.write_parquet(os.path.join(csv_folder_path, file.replace("csv", "parquet")))

In [6]:
combined_df = pl.read_parquet("data/train_dataset.parquet")

In [7]:
combined_df.head(), combined_df.shape

(shape: (5, 503)
 ┌────────┬───────┬─────┬───────────┬───┬─────────────┬─────────────┬─────────────┬─────────────┐
 │ target ┆ smpl  ┆ id  ┆ feature_1 ┆ … ┆ feature_497 ┆ feature_498 ┆ feature_499 ┆ feature_500 │
 │ ---    ┆ ---   ┆ --- ┆ ---       ┆   ┆ ---         ┆ ---         ┆ ---         ┆ ---         │
 │ i64    ┆ str   ┆ i64 ┆ f32       ┆   ┆ f32         ┆ f32         ┆ f32         ┆ f32         │
 ╞════════╪═══════╪═════╪═══════════╪═══╪═════════════╪═════════════╪═════════════╪═════════════╡
 │ 0      ┆ train ┆ 0   ┆ 0.372342  ┆ … ┆ 1.45386     ┆ -0.83348    ┆ 0.184028    ┆ -0.438225   │
 │ 0      ┆ train ┆ 1   ┆ 0.382215  ┆ … ┆ 0.491954    ┆ -0.003686   ┆ 0.46972     ┆ -1.094604   │
 │ 0      ┆ train ┆ 2   ┆ 0.472528  ┆ … ┆ 0.147941    ┆ -0.007553   ┆ -0.981296   ┆ 0.27093     │
 │ 0      ┆ train ┆ 3   ┆ 0.51226   ┆ … ┆ 0.000645    ┆ 0.674715    ┆ 0.803094    ┆ 1.037068    │
 │ 0      ┆ train ┆ 4   ┆ 1.487986  ┆ … ┆ 0.458768    ┆ -0.444558   ┆ -0.33371    ┆ 0.380206    │
 └─

In [ ]:
dataframes = pl.read_parquet("data/train_6.parquet")

In [15]:
for columns in dataframes.columns:
    if dataframes[columns].dtype == pl.Float64:
        dataframes = dataframes.with_columns(dataframes[columns].cast(pl.Float32, strict=False).alias(columns))

In [16]:
combined_df = pl.concat([combined_df, dataframes])

In [17]:
combined_df.shape

(4490468, 503)

In [18]:
combined_df.head()

target,smpl,id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,…,feature_464,feature_465,feature_466,feature_467,feature_468,feature_469,feature_470,feature_471,feature_472,feature_473,feature_474,feature_475,feature_476,feature_477,feature_478,feature_479,feature_480,feature_481,feature_482,feature_483,feature_484,feature_485,feature_486,feature_487,feature_488,feature_489,feature_490,feature_491,feature_492,feature_493,feature_494,feature_495,feature_496,feature_497,feature_498,feature_499,feature_500
i64,str,i64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,"""train""",0,0.372342,1.500852,2.133451,-0.957384,-0.119022,0.12292,0.150993,0.506717,-0.839165,-0.199743,0.216119,1.348401,1.065637,-0.946945,0.499681,-0.264383,0.0,-1.33332,0.443795,0.098801,0.0,0.0,0.015876,-0.021957,0.792371,0.013245,0.0,1.017209,-0.448728,0.492206,0.0,-0.851639,0.936328,0.264994,…,-0.645828,0.434229,1.816138,-1.108658,0.826718,-1.196374,0.24742,0.160462,-0.039017,0.499748,0.159741,0.425718,1.162702,-0.632753,-0.517942,0.449427,0.077546,-1.703537,0.005984,-0.873771,-0.246468,1.4243,0.340689,-0.663166,0.517225,-0.286324,57.0,-1.254497,0.0,0.803635,-1.017931,-0.52113,0.647187,1.45386,-0.83348,0.184028,-0.438225
0,"""train""",1,0.382215,0.962629,-0.192558,-1.019986,-1.330412,-0.100149,-1.131326,-1.173272,-1.790011,0.116708,0.788913,0.720656,-0.914471,-0.766861,1.551133,-0.198539,0.0,-0.71487,-0.869011,-0.743693,0.0,0.0,-0.767664,1.141277,-1.286912,-0.572806,0.0,-1.158312,-2.049119,-0.193137,0.0,-0.554452,0.632568,-1.332586,…,-0.126592,-0.614043,0.021234,-0.334816,-0.055649,1.201724,-0.281092,-1.279436,0.674947,-0.198981,-0.614454,-1.006725,-0.103481,0.778153,-1.090015,0.810727,1.101775,-0.761821,-0.112268,-0.491775,1.130508,-0.989669,1.19114,-0.803186,0.566153,1.284961,1.0,-0.884458,0.0,-1.643735,-1.264842,-1.523788,0.604948,0.491954,-0.003686,0.46972,-1.094604
0,"""train""",2,0.472528,-0.695334,0.537968,-0.033,-0.36485,-0.441849,-0.035953,-0.921158,0.746746,0.477138,1.061066,2.348964,-0.740345,1.007488,0.990622,-1.122783,0.0,0.632436,-0.006833,-0.411626,0.0,0.0,-1.270903,-1.879841,0.62167,-0.688338,0.0,2.160033,-1.6759,-0.082673,0.0,-0.148567,-0.46428,-0.178598,…,0.869195,1.376578,0.750041,1.620765,-0.959123,-0.757372,-1.331277,-0.710401,-1.513987,0.720477,-1.287441,0.10716,1.086501,-0.458318,-0.072571,0.470386,1.170615,0.213452,1.806888,-2.085279,-1.035103,-0.173536,-0.470982,-0.134215,0.129898,-0.146585,2.0,-0.322682,0.0,-0.496765,-0.051931,0.743209,-1.395109,0.147941,-0.007553,-0.981296,0.27093
0,"""train""",3,0.51226,-1.23196,0.555326,1.456647,1.435357,0.168613,-0.628737,0.249797,1.475977,-0.169269,0.51034,0.911377,-0.23678,-0.625305,2.107196,1.295551,0.0,0.649062,-0.117766,-1.508139,0.0,0.0,-0.387628,0.545191,0.14498,1.372529,0.0,0.05125,1.53818,0.840503,0.0,-0.385656,0.915411,-1.591858,…,1.28186,0.303849,0.803946,1.482661,-0.109579,1.002381,-0.432043,-0.381731,0.017506,-0.059549,-1.383963,0.008786,-1.59286,3.101853,0.318749,0.923378,-0.717617,0.383301,-1.171417,0.411504,1.426704,-0.174565,1.742944,0.697967,0.590742,-0.22384,3.0,0.976566,0.0,1.183049,-0.178662,1.468147,-0.791254,0.000645,0.674715,0.803094,1.037068
0,"""train""",4,1.487986,-0.164392,-1.197163,1.549057,0.952673,1.144201,-0.964,0.0319,1.262253,-0.456219,-0.55885,0.035832,-1.898993,0.24009,0.788041,0.478828,1.0,0.30892,-0.966941,-0.525692,0.0,1.0,0.065214,-0.504748,0.022186,0.760587,0.0,2.040007,0.220125,1.336657,0.0,0.360773,0.636532,

In [19]:
combined_df = combined_df.sort(by=["id"])

In [21]:
combined_df.shape

(4490468, 503)

In [74]:
del dataframes

In [73]:
for columns in combined_df.columns:
    if combined_df[columns].dtype == pl.Float64:
        combined_df = combined_df.with_columns(combined_df[columns].cast(pl.Float32, strict=False).alias(columns))

In [22]:
combined_df.write_parquet("data/train_dataset.parquet", compression="zstd", compression_level=17)

#### dfgds

In [2]:
data = pl.read_parquet("data/train_1-002.parquet")
data.head()

target,smpl,id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,…,feature_464,feature_465,feature_466,feature_467,feature_468,feature_469,feature_470,feature_471,feature_472,feature_473,feature_474,feature_475,feature_476,feature_477,feature_478,feature_479,feature_480,feature_481,feature_482,feature_483,feature_484,feature_485,feature_486,feature_487,feature_488,feature_489,feature_490,feature_491,feature_492,feature_493,feature_494,feature_495,feature_496,feature_497,feature_498,feature_499,feature_500
i64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,"""train""",0,0.372342,1.5008518,2.133451,-0.957384,-0.119022,0.1229198,0.150993,0.506717,-0.839165,-0.199743,0.216119,1.3484006,1.065637,-0.946945,0.4996807,-0.264383,0.0,-1.33332,0.443795,0.098801,0.0,0.0,0.015876,-0.021957,0.7923714,0.013245,0.0,1.0172088,-0.448728,0.492206,0.0,-0.851639,0.9363277,0.264994,…,-0.645828,0.4342295,1.8161377,-1.108658,0.8267176,-1.196374,0.24742,0.160462,-0.039017,0.499748,0.159741,0.425718,1.1627017,-0.632753,-0.517942,0.449427,0.077546,-1.703537,0.005984,-0.873771,-0.246468,1.4243001,0.340689,-0.663166,0.5172254,-0.286324,57.0,-1.254497,0.0,0.8036352,-1.017931,-0.52113,0.6471875,1.4538602,-0.83348,0.184028,-0.438225
0,"""train""",1,0.382215,0.9626286,-0.192558,-1.019986,-1.330412,-0.100149,-1.131326,-1.173272,-1.790011,0.116708,0.7889134,0.720656,-0.914471,-0.766861,1.5511327,-0.198539,0.0,-0.71487,-0.869011,-0.743693,0.0,0.0,-0.767664,1.1412772,-1.286912,-0.572805,0.0,-1.158312,-2.04912,-0.193137,0.0,-0.554452,0.6325681,-1.332586,…,-0.126592,-0.614043,0.0212341,-0.334816,-0.055649,1.2017244,-0.281092,-1.279436,0.674947,-0.198981,-0.614454,-1.006725,-0.103481,0.778153,-1.090015,0.810727,1.1017752,-0.761821,-0.112268,-0.491775,1.130508,-0.989669,1.1911404,-0.803186,0.5661534,1.2849605,1.0,-0.884458,0.0,-1.643735,-1.264842,-1.523788,0.604948,0.491954,-0.003686,0.4697199,-1.094604
0,"""train""",2,0.472528,-0.695334,0.537968,-0.033,-0.36485,-0.441849,-0.035953,-0.921158,0.746746,0.4771376,1.0610658,2.3489637,-0.740345,1.007488,0.9906224,-1.122783,0.0,0.632436,-0.006833,-0.411627,0.0,0.0,-1.270903,-1.879841,0.6216702,-0.688338,0.0,2.1600327,-1.6759,-0.082673,0.0,-0.148567,-0.46428,-0.178598,…,0.8691954,1.3765785,0.750041,1.620765,-0.959123,-0.757373,-1.331277,-0.710402,-1.513987,0.720477,-1.287441,0.10716,1.0865015,-0.458318,-0.072571,0.470386,1.1706146,0.213452,1.8068883,-2.085279,-1.035103,-0.173536,-0.470982,-0.134215,0.129898,-0.146585,2.0,-0.322682,0.0,-0.496765,-0.051931,0.7432095,-1.395109,0.147941,-0.007553,-0.981296,0.27093
0,"""train""",3,0.5122596,-1.23196,0.5553261,1.4566467,1.4353575,0.1686132,-0.628737,0.249797,1.4759773,-0.169269,0.5103397,0.911377,-0.23678,-0.625305,2.1071956,1.2955506,0.0,0.6490616,-0.117766,-1.508139,0.0,0.0,-0.387628,0.5451911,0.14498,1.372529,0.0,0.05125,1.5381795,0.8405032,0.0,-0.385656,0.9154105,-1.591858,…,1.2818605,0.303849,0.803946,1.4826615,-0.109579,1.0023811,-0.432043,-0.381731,0.017506,-0.059549,-1.383962,0.008786,-1.59286,3.1018534,0.318749,0.923378,-0.717617,0.383301,-1.171417,0.411504,1.4267044,-0.174565,1.7429444,0.6979669,0.5907423,-0.22384,3.0,0.976566,0.0,1.1830486,-0.178662,1.4681466,-0.791254,0.000645,0.674715,0.803094,1.037068
0,"""train""",4,1.4879864,-0.164392,-1.197163,1.5490566,0.9526726,1.1442006,-0.964,0.0318995,1.262253,-0.456219,-0.55885,0.035832,-1.898993,0.24009,0.7880415,0.478828,1.0,0.3089197,-0.966941,-0.525692,0.0,1.0,0.0652

In [3]:
for columns in data.columns:
    if data[columns].dtype == pl.Float64:
        data = data.with_columns(data[columns].cast(pl.Float32, strict=False).alias(columns))

In [44]:
data = data.drop("smpl")

In [8]:
data.describe()

statistic,target,smpl,id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,…,feature_464,feature_465,feature_466,feature_467,feature_468,feature_469,feature_470,feature_471,feature_472,feature_473,feature_474,feature_475,feature_476,feature_477,feature_478,feature_479,feature_480,feature_481,feature_482,feature_483,feature_484,feature_485,feature_486,feature_487,feature_488,feature_489,feature_490,feature_491,feature_492,feature_493,feature_494,feature_495,feature_496,feature_497,feature_498,feature_499,feature_500
str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",449046.0,"""449046""",449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,…,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0,449046.0
"""null_count""",0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",0.011551,null,224522.5,-0.033924,-0.00611,0.016456,-0.046785,0.050992,0.009004,-0.023415,-0.10847,-0.008339,-0.025386,-0.086228,0.502751,-0.023261,-0.117075,0.009337,-0.063689,0.249179,-0.009792,0.033657,-0.021057,0.1585,0.249179,0.019193,0.003554,0.031133,-0.002671,0.071405,0.024551,-0.037627,-0.032464,0.12227,0.000702,0.052095,…,-0.005741,0.015001,-0.040843,-0.001003,0.000644,-0.003894,-0.135432,-0.017948,-0.002468,-0.062583,0.015995,-0.014824,-0.078167,0.082329,-0.005962,-0.015322,-0.082777,-0.099434,0.055482,-0.025645,-0.011117,0.004557,0.013971,-0.022661,-0.057921,0.00049,14.639001,-0.063176,0.1585,-0.053474,-0.067469,-0.013554,-0.075068,-0.001039,-0.038255,-0.030217,0.046983
"""std""",0.106854,null,129628.558827,0.968003,1.000569,1.01037,0.955292,0.998469,0.98268,0.99169,0.934768,0.946188,1.003378,0.926452,0.959538,0.981401,0.929758,1.005362,0.93798,0.432538,0.928113,0.972302,0.979695,0.36521,0.432538,1.008082,1.011252,1.019362,1.004045,0.2575,1.009855,0.956444,0.955428,0.330542,0.993248,1.012203,…,0.993399,1.011809,0.985931,0.91842,0.999955,0.999613,0.951912,0.986851,0.996148,0.971425,0.993252,0.999466,0.96457,1.044102,0.991202,0.979982,0.939855,0.951247,0.93262,0.969462,0.990618,0.995819,1.005089,0.983804,0.990518,1.000888,15.960107,0.955138,0.36521,0.97386,0.988893,0.945551,0.968957,1.010042,0.962581,1.012119,1.018447
"""min""",0.0,"""train""",0.0,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-3.670716,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,0.0,-5.199337,-5.199337,-5.199337,0.0,0.0,-5.199337,-5.199337,-5.199337,-5.199337,0.0,-5.199337,-5.199337,-5.199337,0.0,-5.199337,-5.199337,…,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-5.199337,-5.19

In [4]:
data.write_parquet("data/test_preprocessing.parquet", compression="zstd", compression_level=17)

In [37]:
data["feature_17"].unique().to_list()

[0, 1]

In [31]:
data["feature_7"].cast(pl.Int64, strict=False).equals(data["feature_7"])

False

In [ ]:
data = data.drop(["smpl", "id"])
data.head()

In [ ]:
data = data.sample(2000)

In [ ]:
data.head().to_pandas(use_pyarrow_extension_array=True)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression()
model

In [ ]:
model.fit(data.drop("target"), data["target"])

In [2]:
data = pl.read_parquet("../data/sub_train_dataset.parquet")
data.head()

target,smpl,id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,…,feature_464,feature_465,feature_466,feature_467,feature_468,feature_469,feature_470,feature_471,feature_472,feature_473,feature_474,feature_475,feature_476,feature_477,feature_478,feature_479,feature_480,feature_481,feature_482,feature_483,feature_484,feature_485,feature_486,feature_487,feature_488,feature_489,feature_490,feature_491,feature_492,feature_493,feature_494,feature_495,feature_496,feature_497,feature_498,feature_499,feature_500
i64,str,i64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
1,"""train""",51,0.575924,0.22133,1.684064,0.108184,0.086583,0.873604,-0.981125,-1.304021,-0.666024,-0.337244,1.180272,0.029739,-1.263854,1.103023,-0.075711,-0.399112,0.0,-0.954556,-0.422704,-0.20565,0.0,0.0,1.995724,1.263619,-0.153048,1.62713,0.0,-0.917502,-1.384319,-0.564511,0.0,-0.17391,0.479087,-0.634089,…,0.046279,-0.013041,0.408524,-1.466392,1.135502,-0.130431,0.846442,0.474579,-0.503552,0.714296,-0.230602,0.313121,1.372373,2.015766,0.743366,0.520015,-0.453539,0.740631,0.028593,-0.574654,-1.433721,-0.804884,0.025428,0.093243,0.347581,-0.030351,1.0,1.325476,0.0,-0.321562,-0.683197,-1.618111,0.509566,-0.890903,0.684632,0.10125,1.468575
1,"""train""",116,0.535891,0.073505,-1.285314,-0.486952,-0.132037,-0.532459,1.687938,-0.875687,-0.172725,-0.965682,0.387123,1.11267,-1.140538,-1.310603,0.421993,-0.417016,0.0,0.221355,0.438114,1.281644,0.0,0.0,0.944637,-0.241344,0.289947,0.793668,0.0,-1.493517,-0.300085,-0.606894,1.0,0.76456,1.395799,0.349063,…,0.716419,0.644251,1.069474,-0.46224,1.294645,0.420474,-0.190662,-0.799477,-0.580747,-0.979771,0.648963,0.488698,-1.702593,1.28907,0.754406,1.444602,0.894355,0.064694,-0.343533,-0.846417,0.694101,1.61691,0.342168,0.267461,0.260128,-0.065663,2.0,-2.057568,0.0,1.48217,1.710262,-0.172929,0.447836,-0.400597,1.372346,-0.339095,-1.206653
1,"""train""",129,0.907321,-0.670663,-0.181707,0.160142,1.15332,0.675533,-0.601514,-1.03188,0.447099,0.933306,1.049696,-0.143409,0.289074,1.063375,0.604891,0.077759,0.0,0.238669,1.543897,-0.088067,0.0,0.0,0.216293,-1.576575,0.553923,0.13333,0.0,-1.575375,-0.084472,-0.122795,0.0,1.363174,-0.499273,0.416601,…,-0.19352,0.045254,1.221884,-0.287202,0.454189,1.599415,-0.26421,1.300249,-0.385706,2.186214,0.409925,-1.327251,-0.837308,0.906037,-0.472586,0.060089,1.164486,1.20469,0.169166,-0.126535,1.888815,-0.910683,-0.041059,-0.372387,0.771838,-0.035046,15.0,0.414452,0.0,0.840254,1.885963,0.440201,0.595312,0.081776,-0.911395,0.250122,-0.02454
1,"""train""",391,1.595058,-0.678757,0.135097,-0.531307,0.167596,-0.666086,-0.018718,-1.951071,-0.225989,-0.490973,0.380046,1.489469,0.780963,0.583641,-1.01475,-0.38029,0.0,-0.102469,0.169021,0.458915,0.0,0.0,-0.892611,-1.143362,-0.093893,-2.69318,0.0,0.229763,-1.485948,-1.20618,0.0,0.440022,1.3,0.92657,…,-1.187309,-0.219683,0.526998,-0.456207,-1.948905,-2.877563,-1.360259,0.094024,1.522478,0.756947,0.321435,0.790942,1.420553,1.131217,0.750021,-0.831505,0.409404,-0.164877,0.479127,-0.142328,-0.129153,-2.353522,0.899784,1.054675,-0.423816,0.593567,2.0,-0.818523,0.0,-0.392109,-0.234374,-0.232281,-0.158962,0.223449,0.989992,0.579262,-1.272855
1,"""train""",417,-0.633544,-0.810396,0.430721,-0.298063,2.619097,2.096483,-0.01499,0.592974,-1.098315,-2.06642,0.305547,0.536667,1.02318,0.158979,1.04366,0.134349,0.0,-0.878825,0.181724,1.541455,0.0,0.0,1.689923,-1.739879,-1.05516,1.458465,0.0,-0.96353,-0.274656,-0.146203,1.0,0.561834,-0.2

In [3]:
from clearml import Task
import pickle

# Подключение к задаче ClearML, которая содержит вашу модель
task = Task.get_task(project_name="Alfa_hack", task_name="Blending Catboost train full_dataset 07.11.24 v1.0")

In [4]:
my_local_copy_of_the_previously_uploaded_artifact = task.artifacts["catboost_weights"].get_local_copy()

In [5]:
with open(my_local_copy_of_the_previously_uploaded_artifact, "rb") as file:
    model = pickle.load(file)


In [8]:
model.predict(data.drop(["target", "smpl", "id"]).sample(1).to_pandas(use_pyarrow_extension_array=True))

CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=30]=0.0 : cat_features must be integer or string, real number values and NaN values should be converted to string.